## New York flights analysis 2013
### Qestions to be answered by this case study:
1. What is the most popular destination city from New York?
2. Which month is the busiest of the year?
3. Which airline is the most punctual?
4. What destination is the longest duration?
5. What airline is the worst in terms of delays?
6. Which airline has the highest capacity of seats?
7. Which airplane model is the highest in use and from which manufacturer?

In [3]:
# Step 1: Reading the data and importing packages
import os
import pandas as pd
import numpy as np

path = 'C:/Users/igorr/Documents/Python Scripts/retail-analytics-trade-modeling/resources'
file_names = ['airlines.csv', 'airports.csv', 'flights.csv', 'planes.csv']

# Create a dictionary to store the dataframes
dataframes = {}

# Loop through the file names and read the CSV files
for file_name in file_names:
    file_path = os.path.join(path, file_name)
    dataframes[file_name.replace('.csv', '')] = pd.read_csv(file_path)

# Access the dataframes using their names
airlines = dataframes['airlines']
airports = dataframes['airports']
flights = dataframes['flights']
planes = dataframes['planes']


### What is the most popular destination city from New York?
Major airports in the NYC area are:
1. LaGuardia Airport (LGA)
2. John F. Kennedy International Airport (JFK)
3. Newark Liberty International Airport (EWR)

source: https://www.afar.com/magazine/the-ultimate-guide-to-new-york-citys-airports

In [13]:
# Looking at the flights dataframe
flights.head()

# in the df there are columns origin and destination which contain airport codes

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
0,2013,1,1,517.0,515,2.0,830.0,819,11.0,UA,1545,N14228,EWR,IAH,227.0,1400,5,15,2013-01-01T10:00:00Z
1,2013,1,1,533.0,529,4.0,850.0,830,20.0,UA,1714,N24211,LGA,IAH,227.0,1416,5,29,2013-01-01T10:00:00Z
2,2013,1,1,542.0,540,2.0,923.0,850,33.0,AA,1141,N619AA,JFK,MIA,160.0,1089,5,40,2013-01-01T10:00:00Z
3,2013,1,1,544.0,545,-1.0,1004.0,1022,-18.0,B6,725,N804JB,JFK,BQN,183.0,1576,5,45,2013-01-01T10:00:00Z
4,2013,1,1,554.0,600,-6.0,812.0,837,-25.0,DL,461,N668DN,LGA,ATL,116.0,762,6,0,2013-01-01T11:00:00Z


In [9]:
# Looking at airports dataframe - we're looking for LGA, JFK and EWR
airports.head()

,faa,name,lat,lon,alt,tz,dst,tzone
0,04G,Lansdowne Airport,41.130472,-80.619583,1044,-5,A,America/New_York
1,06A,Moton Field Municipal Airport,32.460572,-85.680028,264,-6,A,America/Chicago
2,06C,Schaumburg Regional,41.989341,-88.101243,801,-6,A,America/Chicago
3,06N,Randall Airport,41.431912,-74.391561,523,-5,A,America/New_York
4,09J,Jekyll Island Airport,31.074472,-81.427778,11,-5,A,America/New_York


In [17]:
### I want to confirm that indeed, faa column contains airport codes in the NYC area
# creating a list of airports of interest
nyc_airports = ['LGA', 'JFK', 'EWR']

# filtering the airports df
airports_nyc = airports[airports['faa'].isin(airport_codes)]

# display the filtered df
airports_nyc

,faa,name,lat,lon,alt,tz,dst,tzone
460,EWR,Newark Liberty Intl,40.692500,-74.168667,18,-5,A,America/New_York
691,JFK,John F Kennedy Intl,40.639751,-73.778925,13,-5,A,America/New_York
786,LGA,La Guardia,40.777245,-73.872608,22,-5,A,America/New_York


In [22]:
# Filtering fights originating from New York City (JFK, LGA, EWR)
nyc_flights = flights[flights['origin'].isin(nyc_airports)]
nyc_flights.head()

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
0,2013,1,1,517.0,515,2.0,830.0,819,11.0,UA,1545,N14228,EWR,IAH,227.0,1400,5,15,2013-01-01T10:00:00Z
1,2013,1,1,533.0,529,4.0,850.0,830,20.0,UA,1714,N24211,LGA,IAH,227.0,1416,5,29,2013-01-01T10:00:00Z
2,2013,1,1,542.0,540,2.0,923.0,850,33.0,AA,1141,N619AA,JFK,MIA,160.0,1089,5,40,2013-01-01T10:00:00Z
3,2013,1,1,544.0,545,-1.0,1004.0,1022,-18.0,B6,725,N804JB,JFK,BQN,183.0,1576,5,45,2013-01-01T10:00:00Z
4,2013,1,1,554.0,600,-6.0,812.0,837,-25.0,DL,461,N668DN,LGA,ATL,116.0,762,6,0,2013-01-01T11:00:00Z


In [34]:
# Group the df by destination airport and count the flights
destination_counts = nyc_flights['dest'].value_counts().reset_index()
destination_counts.columns = ['Destination', 'Flights Count']

# adding airport names
destination_counts = destination_counts.merge(airports[['faa', 'name']], left_on = 'Destination', right_on = 'faa')
destination_counts = destination_counts.drop(columns = ['faa'])

destination_counts.head()


,Destination,Flights Count,name
0,ORD,17283,Chicago Ohare Intl
1,ATL,17215,Hartsfield Jackson Atlanta Intl
2,LAX,16174,Los Angeles Intl
3,BOS,15508,General Edward Lawrence Logan Intl
4,MCO,14082,Orlando Intl


In [36]:
# sort the destination by flights count in descending order
sorted_destinations = destination_counts.sort_values(by = 'Flights Count', ascending = False)

# get the most popular destinations
most_popular_destination = sorted_destinations.iloc[0]
most_popular_destination

Destination                     ORD
Flights Count                 17283
name             Chicago Ohare Intl
Name: 0, dtype: object

In [40]:
# display the most popular destinations
print("Most popular destination from New York City is:", most_popular_destination['name'], ", code:", most_popular_destination['Destination'])
print("Number of flights to the destination:", most_popular_destination['Flights Count'])

Most popular destination from New York City is: Chicago Ohare Intl , code: ORD
Number of flights to the destination: 17283


### Which month is the busiest of the year?
Assuming the busiest month meaning there are the most flights in this month.
`flight` column in flights dataframe seems like a good candidate as it contains flight numbers

In [41]:
# checking out the datatypes in flights df
flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 336776 entries, 0 to 336775
Data columns (total 19 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   year            336776 non-null  int64  
 1   month           336776 non-null  int64  
 2   day             336776 non-null  int64  
 3   dep_time        328521 non-null  float64
 4   sched_dep_time  336776 non-null  int64  
 5   dep_delay       328521 non-null  float64
 6   arr_time        328063 non-null  float64
 7   sched_arr_time  336776 non-null  int64  
 8   arr_delay       327346 non-null  float64
 9   carrier         336776 non-null  object 
 10  flight          336776 non-null  int64  
 11  tailnum         334264 non-null  object 
 12  origin          336776 non-null  object 
 13  dest            336776 non-null  object 
 14  air_time        327346 non-null  float64
 15  distance        336776 non-null  int64  
 16  hour            336776 non-null  int64  
 17  minute    

In [42]:
# checking total null values in `flight` column
flights.flight.isnull().sum()

0

In [52]:
# check and remove duplicates from `flight` column
flights.drop_duplicates(subset = ['flight'], inplace = True)

# group by month and count the number of flights in each
monthly_flight_counts = flights.groupby('month')['flight'].count()
monthly_flight_counts

month
1     1652
2      218
3      180
4      119
5      102
6       89
7       78
8       91
9       47
10     561
11     369
12     338
Name: flight, dtype: int64

In [53]:
# find the busiest month
busiest_month = monthly_flight_counts.idxmax()

print("The busiest month of the year is:", busiest_month)

The busiest month of the year is: 1


### What destination is the longest duration?

In [55]:
# peeking at the flights df again
flights.head()

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
0,2013,1,1,517.0,515,2.0,830.0,819,11.0,UA,1545,N14228,EWR,IAH,227.0,1400,5,15,2013-01-01T10:00:00Z
1,2013,1,1,533.0,529,4.0,850.0,830,20.0,UA,1714,N24211,LGA,IAH,227.0,1416,5,29,2013-01-01T10:00:00Z
2,2013,1,1,542.0,540,2.0,923.0,850,33.0,AA,1141,N619AA,JFK,MIA,160.0,1089,5,40,2013-01-01T10:00:00Z
3,2013,1,1,544.0,545,-1.0,1004.0,1022,-18.0,B6,725,N804JB,JFK,BQN,183.0,1576,5,45,2013-01-01T10:00:00Z
4,2013,1,1,554.0,600,-6.0,812.0,837,-25.0,DL,461,N668DN,LGA,ATL,116.0,762,6,0,2013-01-01T11:00:00Z


In [69]:
# looks like air_time is the best candidate to answer this question as it indicates the amont of time spent in air, in minutes
max_air_time_minutes = flights['air_time'].max()

# converting the maximum from minutes to hours
max_air_time_hours = max_air_time_minutes // 60
remaining_minutes = max_air_time_minutes % 60

In [70]:
# locate the row(s) with the maximum air time
longest_duration_flight = flights[flights['air_time'] == max_air_time_minutes]
longest_duration_flight

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
162,2013,1,1,857.0,900,-3.0,1516.0,1530,-14.0,HA,51,N380HA,JFK,HNL,659.0,4983,9,0,2013-01-01T14:00:00Z


In [72]:
# retrieve the destination and origin airports with the longest duration
longest_duration_origin = longest_duration_flight['origin'].values[0]
longest_duration_destination = longest_duration_flight['dest'].values[0]

In [73]:
# print the results
print("The flight with the longest duration is from", longest_duration_origin, "to", longest_duration_destination)
print("Maximum Air Time:", max_air_time_hours, "hours", remaining_minutes, "minutes")

The flight with the longest duration is from JFK to HNL
Maximum Air Time: 10.0 hours 59.0 minutes


### What airline is the worst in terms of delays?
carrier, dep_delay and arr_delay columns from `flights` df will be useful here. Negative values in dep_delay and arr_delay columns mean that the flight departed or arrived earlier than scheduled.
Let's assume that the worst carrier in terms of delays is that which has the highest average dep_delay time **excluding** negative values in dep_delay

In [74]:
# Filter out rows with negative dep_delay and arr_delay values
filtered_flights = flights[(flights['dep_delay'] >= 0) & (flights['arr_delay'] >= 0)]
filtered_flights['dep_delay'].describe()

count    1142.000000
mean       38.012259
std        54.318484
min         0.000000
25%         7.000000
50%        21.000000
75%        48.750000
max       853.000000
Name: dep_delay, dtype: float64

In [75]:
# looking if the filtering has worked as expected - min should be = 0
filtered_flights['arr_delay'].describe()

count    1142.000000
mean       39.196147
std        53.986268
min         0.000000
25%         9.000000
50%        23.000000
75%        47.000000
max       851.000000
Name: arr_delay, dtype: float64

In [76]:
# calculating the average dep_delay for each carrier
average_dep_delay_by_carried = filtered_flights.groupby('carrier')['dep_delay'].mean()
average_dep_delay_by_carried

carrier
9E     51.745763
AA     31.000000
B6     25.268657
DL     33.602941
EV     51.049275
F9    191.000000
FL     16.750000
MQ     53.909091
OO     34.000000
UA     26.883162
US     22.156250
VX      0.000000
WN     25.529412
YV     28.250000
Name: dep_delay, dtype: float64

In [77]:
# calculating the average arr_delay for each carrier
average_arr_delay_by_carried = filtered_flights.groupby('carrier')['arr_delay'].mean()
average_arr_delay_by_carried

carrier
9E     47.745763
AA     38.137931
B6     28.208955
DL     33.558824
EV     52.031884
F9    187.000000
FL     23.750000
MQ     60.318182
OO     55.000000
UA     29.460481
US     22.031250
VX      9.000000
WN     25.426471
YV     27.000000
Name: arr_delay, dtype: float64

In [79]:
# finding the carrier with the highest dep_delay
max_dep_delay_carrier = average_dep_delay_by_carried.idxmax()

# finding the carrier with the highest arr_delay
max_arr_delay_carrier = average_arr_delay_by_carried.idxmax()

In [80]:
# print the results
print("The worst airline in terms of average departure delay is:", max_dep_delay_carrier)
print("The worst airline in terms of average arrival delay is:", max_arr_delay_carrier)

The worst airline in terms of average departure delay is: F9
The worst airline in terms of average arrival delay is: F9


### Which airline has the highest capacity of seats?
Let's look at the airlines and planes dataframes in order to see if there are any columns that will aid this part of the analysis.
Let's also assume that the top 1 carrier in terms of capacity is the one that has the highest seat average across all its planes.

In [81]:
# peeking at the airlines df
airlines.head()

,carrier,name
0,9E,Endeavor Air Inc.
1,AA,American Airlines Inc.
2,AS,Alaska Airlines Inc.
3,B6,JetBlue Airways
4,DL,Delta Air Lines Inc.


In [82]:
# peeking at planes df
planes.head()

,tailnum,year,type,manufacturer,model,engines,seats,speed,engine
0,N10156,2004.0,Fixed wing multi engine,EMBRAER,EMB-145XR,2,55,NaN,Turbo-fan
1,N102UW,1998.0,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2,182,NaN,Turbo-fan
2,N103US,1999.0,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2,182,NaN,Turbo-fan
3,N104UW,1999.0,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2,182,NaN,Turbo-fan
4,N10575,2002.0,Fixed wing multi engine,EMBRAER,EMB-145LR,2,55,NaN,Turbo-fan


In [83]:
# looks like planes df has seats column that will be perfect to see the highest capacity and also tailnum column that will allow us to join to flights in order to discover the carrier owning the plane
flights.head()

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
0,2013,1,1,517.0,515,2.0,830.0,819,11.0,UA,1545,N14228,EWR,IAH,227.0,1400,5,15,2013-01-01T10:00:00Z
1,2013,1,1,533.0,529,4.0,850.0,830,20.0,UA,1714,N24211,LGA,IAH,227.0,1416,5,29,2013-01-01T10:00:00Z
2,2013,1,1,542.0,540,2.0,923.0,850,33.0,AA,1141,N619AA,JFK,MIA,160.0,1089,5,40,2013-01-01T10:00:00Z
3,2013,1,1,544.0,545,-1.0,1004.0,1022,-18.0,B6,725,N804JB,JFK,BQN,183.0,1576,5,45,2013-01-01T10:00:00Z
4,2013,1,1,554.0,600,-6.0,812.0,837,-25.0,DL,461,N668DN,LGA,ATL,116.0,762,6,0,2013-01-01T11:00:00Z


In [86]:
# isolating relevant columns
flights_relevant = flights[['tailnum', 'carrier']]
planes_relevant = planes[['tailnum', 'seats']]

# merging dataframes:
planes_carriers = pd.merge(flights_relevant, planes_relevant, on = 'tailnum', how = 'left')
planes_carriers

,tailnum,carrier,seats
0,N14228,UA,149.0
1,N24211,UA,149.0
2,N619AA,AA,178.0
3,N804JB,B6,200.0
4,N668DN,DL,178.0
...,...,...,...
3839,N11192,EV,55.0
3840,N8412F,9E,55.0
3841,N8554A,9E,55.0
3842,N805AY,9E,55.0


In [88]:
# grouping by carrier and calculating average seats
carrier_avg_seats = planes_carriers.groupby('carrier')['seats'].mean().reset_index()
carrier_avg_seats

,carrier,seats
0,9E,76.015228
1,AA,163.214286
2,AS,149.000000
3,B6,131.253165
4,DL,161.084337
5,EV,63.708251
6,F9,182.000000
7,FL,102.722222
8,HA,377.000000
9,MQ,11.250000


In [91]:
# find the top carrier by average seats
top_carrier = carrier_avg_seats.sort_values('seats', ascending = False).head(1)
top_carrier

,carrier,seats
8,HA,377.0


### Which airplane model is the highest in use and from which manufacturer?
Let's assume that the airplane model in the highest use is that which is used in the most flights. 

In [92]:
# first let's take a look at flights and planes dataframes
planes.head()

# planes contain the manufacturer column which we will use in the analysis

,tailnum,year,type,manufacturer,model,engines,seats,speed,engine
0,N10156,2004.0,Fixed wing multi engine,EMBRAER,EMB-145XR,2,55,NaN,Turbo-fan
1,N102UW,1998.0,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2,182,NaN,Turbo-fan
2,N103US,1999.0,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2,182,NaN,Turbo-fan
3,N104UW,1999.0,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2,182,NaN,Turbo-fan
4,N10575,2002.0,Fixed wing multi engine,EMBRAER,EMB-145LR,2,55,NaN,Turbo-fan


In [94]:
flights.head()

# looks like tailnum will be helpful again

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
0,2013,1,1,517.0,515,2.0,830.0,819,11.0,UA,1545,N14228,EWR,IAH,227.0,1400,5,15,2013-01-01T10:00:00Z
1,2013,1,1,533.0,529,4.0,850.0,830,20.0,UA,1714,N24211,LGA,IAH,227.0,1416,5,29,2013-01-01T10:00:00Z
2,2013,1,1,542.0,540,2.0,923.0,850,33.0,AA,1141,N619AA,JFK,MIA,160.0,1089,5,40,2013-01-01T10:00:00Z
3,2013,1,1,544.0,545,-1.0,1004.0,1022,-18.0,B6,725,N804JB,JFK,BQN,183.0,1576,5,45,2013-01-01T10:00:00Z
4,2013,1,1,554.0,600,-6.0,812.0,837,-25.0,DL,461,N668DN,LGA,ATL,116.0,762,6,0,2013-01-01T11:00:00Z


In [95]:
# merging on tailnum
flights_planes_models = pd.merge(flights[['tailnum', 'flight']], planes[['tailnum', 'manufacturer', 'model']], on = 'tailnum', how = 'inner')
flights_planes_models

,tailnum,flight,manufacturer,model
0,N14228,1545,BOEING,737-824
1,N14228,1572,BOEING,737-824
2,N24211,1714,BOEING,737-824
3,N24211,1440,BOEING,737-824
4,N24211,1742,BOEING,737-824
...,...,...,...,...
3549,N8554A,3583,BOMBARDIER INC,CL-600-2B19
3550,N237WN,113,BOEING,737-7H4
3551,N437WN,154,BOEING,737-7H4
3552,N8533D,3855,BOMBARDIER INC,CL-600-2B19


In [96]:
# count the number of flights for each model
model_use_cnt = flights_planes_models.groupby(['manufacturer', 'model']).size().reset_index(name = 'flight_count')
model_use_cnt

,manufacturer,model,flight_count
0,AIRBUS,A319-112,1
1,AIRBUS,A319-114,32
2,AIRBUS,A319-131,8
3,AIRBUS,A320-211,2
4,AIRBUS,A320-212,3
...,...,...,...
73,MCDONNELL DOUGLAS AIRCRAFT CO,MD-88,73
74,MCDONNELL DOUGLAS CORPORATION,MD-88,12
75,PAIR MIKE E,FALCON XP,1
76,PIPER,PA-32R-300,1


In [97]:
# find the highest in use
highest_in_use = model_use_cnt.sort_values('flight_count', ascending=False).head(1)

print(highest_in_use)

   manufacturer      model  flight_count
63      EMBRAER  EMB-145LR           443
